<a href="https://colab.research.google.com/github/jyryu3161/bio_system_design/blob/main/Model_evaluation_essentiality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cobra

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.8/141.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.1/753.1 kB 52.1 MB/s eta 0:00:00


In [4]:
!git clone https://github.com/jyryu3161/bio_system_design.git

Cloning into 'bio_system_design'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 46 (delta 16), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (46/46), 1.03 MiB | 3.82 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [3]:
import cobra
from cobra.io import load_model

model = load_model("iML1515")

2712


In [ ]:
print(len(model.reactions))


In [22]:
from cobra.io import load_model
from cobra.flux_analysis.moma import moma
import tqdm

def evaluate_essentiality_prediction(exp_data, pred_data, common_genes):
    # Confusion matrix 요소 계산
    TP = 0  # True Positive: 실제 essential, 예측 essential
    TN = 0  # True Negative: 실제 non-essential, 예측 non-essential
    FP = 0  # False Positive: 실제 non-essential, 예측 essential
    FN = 0  # False Negative: 실제 essential, 예측 non-essential

    for gene in common_genes:
        exp_essential = exp_data[gene]
        pred_essential = pred_data[gene]

        if exp_essential == 1 and pred_essential == 1:
            TP += 1
        elif exp_essential == 0 and pred_essential == 0:
            TN += 1
        elif exp_essential == 0 and pred_essential == 1:
            FP += 1
        elif exp_essential == 1 and pred_essential == 0:
            FN += 1

    # 성능 지표 계산
    total = TP + TN + FP + FN

    # Accuracy: (TP + TN) / Total
    accuracy = (TP + TN) / total if total > 0 else 0

    # Sensitivity (Recall, True Positive Rate): TP / (TP + FN)
    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0

    # Specificity (True Negative Rate): TN / (TN + FP)
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0

    # Precision: TP / (TP + FP)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0

    # F1 Score: 2 * (Precision * Recall) / (Precision + Recall)
    f1_score = 2 * (precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0

    results = {
        'TP': TP,
        'TN': TN,
        'FP': FP,
        'FN': FN,
        'Total': total,
        'Accuracy': accuracy,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'Precision': precision,
        'F1_Score': f1_score
    }

    return results

exp_essentiality_info = {}
with open('./bio_system_design/glc_ecoli_essentiality.txt') as fp:
    fp.readline()
    for line in fp:
        sptlist = line.strip().split('\t')
        gene = sptlist[0].strip()
        essentiality = int(sptlist[1].strip())
        exp_essentiality_info[gene] = essentiality

# 1) WT 모델과 WT FBA 해 구하기
wt_solution = model.optimize()           # <- 참조 flux (Solution)
print('WT Growth rate: ', wt_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])
insilico_essentiality_info = {}

# 2) KO 모델 만들기 & FBA 해 구하기
for each_gene in tqdm.tqdm(exp_essentiality_info):
    ko_model = model.copy()
    try:
        ko_model.genes.get_by_id(each_gene).knock_out()
    except:
        continue

    mutant_solution = ko_model.optimize()
    mutant_growth = mutant_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']
    print('Growth rate after %s knock-out : %s'%(each_gene, mutant_growth))

    if mutant_growth < 0.5:
        insilico_essentiality_info[each_gene] = 1
    else:
        insilico_essentiality_info[each_gene] = 0

    if len(insilico_essentiality_info) == 10:
        break

# 실험 결과와 시뮬레이션 결과 모두 존재하는 유전자 선별
common_genes = list(set(set(exp_essentiality_info.keys()) & set(insilico_essentiality_info.keys())))

performance = evaluate_essentiality_prediction(
    exp_essentiality_info,
    insilico_essentiality_info,
    common_genes
)

# 결과 출력
print("\n" + "="*60)
print("Gene Essentiality Prediction Performance")
print("="*60)
print(f"Total genes evaluated: {performance['Total']}")
print(f"\nConfusion Matrix:")
print(f"  True Positive (TP):  {performance['TP']}")
print(f"  True Negative (TN):  {performance['TN']}")
print(f"  False Positive (FP): {performance['FP']}")
print(f"  False Negative (FN): {performance['FN']}")

print(f"\nPerformance Metrics:")
print(f"  Accuracy:    {performance['Accuracy']:.4f} ({performance['Accuracy']*100:.2f}%)")
print(f"  Sensitivity: {performance['Sensitivity']:.4f} ({performance['Sensitivity']*100:.2f}%)")
print(f"  Specificity: {performance['Specificity']:.4f} ({performance['Specificity']*100:.2f}%)")
print(f"  Precision:   {performance['Precision']:.4f} ({performance['Precision']*100:.2f}%)")
print(f"  F1 Score:    {performance['F1_Score']:.4f}")
print("="*60)


WT Growth rate:  0.8769972144269834


  0%|          | 1/1354 [00:00<20:52,  1.08it/s]

Growth rate after b0002 knock-out : 0.8769972144269725


  0%|          | 2/1354 [00:01<18:41,  1.21it/s]

Growth rate after b0003 knock-out : 2.4091278452587314e-16


...

Growth rate after b0025 knock-out : -7.949319822096583e-17


  1%|          | 10/1354 [00:10<30:11,  1.35s/it]

Growth rate after b0029 knock-out : 1.6896253882630634e-16


  1%|          | 10/1354 [00:11<25:54,  1.16s/it]

Growth rate after b0030 knock-out : 0.8769972144269725

Gene Essentiality Prediction Performance
Total genes evaluated: 10

Confusion Matrix:
  True Positive (TP):  4
  True Negative (TN):  5
  False Positive (FP): 0
  False Negative (FN): 1

Performance Metrics:
  Accuracy:    0.9000 (90.00%)
  Sensitivity: 0.8000 (80.00%)
  Specificity: 1.0000 (100.00%)
  Precision:   1.0000 (100.00%)
  F1 Score:    0.8889
